In [126]:
#This is the start of our new project
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [122]:
df1 = pd.read_csv('data/Training_Set_Labels_Y_variable.csv')
df2 = pd.read_csv('data/Training_Set_Values_X_variables.csv')

In [79]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [53]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [54]:
df1["status_group"].value_counts(normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [55]:
df1["status_group"] = df1["status_group"].replace({"non functional": "needs repair", "functional needs repair": "needs repair"})
df1["status_group"].value_counts()

functional      32259
needs repair    27141
Name: status_group, dtype: int64

In [123]:
df1

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


In [124]:
df2

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,3/14/2011,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,3/6/2013,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2/25/2013,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,1/28/2013,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,7/13/2011,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,5/3/2013,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,5/7/2011,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,4/11/2011,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,3/8/2011,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [58]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [59]:
df2["source"].value_counts()
#possible feature

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [60]:
df2["source"].isnull().sum()

0

In [61]:
df2["region"].value_counts()
#possible feature

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [62]:
df2["region"].isnull().sum()

0

In [63]:
df2["management"].value_counts()
#Possible feature

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [64]:
df2["management"].isnull().sum()

0

In [65]:
df2 = df2.loc[df2["construction_year"] != 0]
df2["construction_year"].value_counts()
#Possible feature

2010    2645
2008    2613
2009    2533
2000    2091
2007    1587
2006    1471
2003    1286
2011    1256
2004    1123
2012    1084
2002    1075
1978    1037
1995    1014
2005    1011
1999     979
1998     966
1990     954
1985     945
1996     811
1980     811
1984     779
1982     744
1994     738
1972     708
1974     676
1997     644
1992     640
1993     608
2001     540
1988     521
1983     488
1975     437
1986     434
1976     414
1970     411
1991     324
1989     316
1987     302
1981     238
1977     202
1979     192
1973     184
2013     176
1971     145
1960     102
1967      88
1963      85
1968      77
1969      59
1964      40
1962      30
1961      21
1965      19
1966      17
Name: construction_year, dtype: int64

In [66]:
df2["construction_year"].isnull().sum()

0

In [117]:

# Encode categorical features as dummy variables
encoder = OneHotEncoder(drop='first', sparse=False)
X_cat = encoder.fit_transform(df2[['source', 'region', 'management']])

# Combine dummy variables with other numeric features
X = pd.concat([pd.DataFrame(X_cat), df2['construction_year']], axis=1)

# Target variable
y = df1['status_group']

# Verify the number of samples in X and y
print(f"Number of samples in X: {X.shape[0]}")
print(f"Number of samples in y: {y.shape[0]}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and fit the logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Evaluate the model
accuracy = model.score(X_test_scaled, y_test)
print(f"Model accuracy: {accuracy:.2f}")

Number of samples in X: 59400
Number of samples in y: 59400
Model accuracy: 0.59


In [110]:
# Create a binary target variable (0 for 'functional', 1 for 'needs repair')
le = LabelEncoder()
df1['target'] = le.fit_transform(df1['status_group'])

# Select relevant features (X) and target variable (y)X_cat = encoder.fit_transform(df2[['source', 'region', 'management']])
X_cat = encoder.fit_transform(df2[['source', 'region', 'management']])
X = X_cat
y = df1['target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the multiple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model (you can use R-squared or other metrics)
r_squared = model.score(X_test, y_test)
print(f"R-squared: {r_squared:.2f}")

R-squared: 0.08


In [99]:
# Create a binary target variable (0 for 'functional', 1 for 'needs repair')
le = LabelEncoder()
df1['target'] = le.fit_transform(df1['status_group'])

# Select relevant features (X) and target variable (y)
X_cat = encoder.fit_transform(df2[['source', 'region', 'management']])
X = X_cat
y = df1['target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the decision tree classifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_test_predictions = model.predict(X_test)

# Compute precision, recall, and F1-score
precision1 = precision_score(y_test, y_test_predictions, average='weighted')
recall1 = recall_score(y_test, y_test_predictions, average='weighted')
f1score1 = f1_score(y_test, y_test_predictions, average='weighted')

print(f"Precision: {precision1:.2f}")
print(f"Recall: {recall1:.2f}")
print(f"F1 Score: {f1score1:.2f}")

Precision: 0.59
Recall: 0.61
F1 Score: 0.58


In [100]:
precision2 = precision_score(y_test, y_test_predictions, average='micro')
recall2 = recall_score(y_test, y_test_predictions, average='micro')
f1score2 = f1_score(y_test, y_test_predictions, average='micro')

print(f"Precision: {precision2:.2f}")
print(f"Recall: {recall2:.2f}")
print(f"F1 Score: {f1score2:.2f}")

Precision: 0.61
Recall: 0.61
F1 Score: 0.61


In [101]:
precision3 = precision_score(y_test, y_test_predictions, average='macro')
recall3 = recall_score(y_test, y_test_predictions, average='macro')
f1score3 = f1_score(y_test, y_test_predictions, average='macro')

print(f"Precision: {precision3:.2f}")
print(f"Recall: {recall3:.2f}")
print(f"F1 Score: {f1score3:.2f}")

Precision: 0.53
Recall: 0.44
F1 Score: 0.44


In [102]:
# Create a binary target variable (0 for 'functional', 1 for 'needs repair')
le = LabelEncoder()
df1['target'] = le.fit_transform(df1['status_group'])

# Split data into features (X) and target (y)
X_cat = encoder.fit_transform(df2[['source', 'region', 'management']])
X = X_cat
y = df1['target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision4 = precision_score(y_test, y_pred, average='weighted')
recall4 = recall_score(y_test, y_pred, average='weighted')
f1score4 = f1_score(y_test, y_pred, average='weighted')

print(f"Random Forest Model Accuracy: {accuracy:.2f}")
print(f"Random Forest Model Precision: {precision4:.2f}")
print(f"Random Forest Model Recall: {recall4:.2f}")
print(f"Random Forest Model f1 Score: {f1score4:.2f}")

Random Forest Model Accuracy: 0.61
Random Forest Model Precision: 0.59
Random Forest Model Recall: 0.61
Random Forest Model f1 Score: 0.59


In [103]:
precision5 = precision_score(y_test, y_pred, average='micro')
recall5 = recall_score(y_test, y_pred, average='micro')
f1score5 = f1_score(y_test, y_pred, average= 'micro')

print(f"Random Forest Model Precision: {precision5:.2f}")
print(f"Random Forest Model Recall: {recall5:.2f}")
print(f"Random Forest Model f1 Score: {f1score5:.2f}")

Random Forest Model Precision: 0.61
Random Forest Model Recall: 0.61
Random Forest Model f1 Score: 0.61


In [128]:
#First visual model 
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(X_train, y_train)
dummy_model.predict(X_train)[:50]


array(['functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional'], dtype='<U10')